In [ ]:
import requests
import pandas as pd
import time
import re

LISTING_URL = "https://gateway.chotot.com/v1/public/ad-listing"
DETAIL_URL = "https://gateway.chotot.com/v1/public/ad-listing/{}"


HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Origin": "https://www.nhatot.com",
    "Referer": "https://www.nhatot.com/",
    "cookie": "__cf_bm=PASTE_YOUR_COOKIE_HERE"
}

# Thuê phòng trọ tại Đà Nẵng
PARAMS_BASE = {
    "region_v2": 3017,  # Đà Nẵng
    "cg": 1050,         # Cho thuê phòng trọ
    "st": "u,h",        # trạng thái tin
    "limit": 50,        # số tin mỗi trang
}

# ------------------ HÀM PHỤ TRỢ ------------------
def detect_features(text):
    """Trích xuất các tiện ích từ mô tả"""
    if not text:
        return {"Máy giặt": 0, "Tủ lạnh": 0, "Wifi": 0, "Điều hòa": 0}
    t = text.lower()
    return {
        "Máy giặt": 1 if re.search(r"máy[\s\-]?giặt|may giat|giặt", t) else 0,
        "Tủ lạnh": 1 if re.search(r"tủ[\s\-]?lạnh|tu lanh|tulanh", t) else 0,
        "Wifi": 1 if re.search(r"wi[-\s]?fi|wifi", t) else 0,
        "Điều hòa": 1 if re.search(r"điều[\s\-]?hòa|dieu hoa|máy[\s\-]?lạnh|đh\b", t) else 0,
    }

def safe_get_detail(ad_id, headers, max_retries=3):
    """Lấy chi tiết tin đăng (mô tả + tham số)"""
    backoff = 0.5
    for _ in range(max_retries):
        try:
            r = requests.get(DETAIL_URL.format(ad_id), headers=headers, timeout=10)
            if r.status_code == 200:
                d = r.json()
                params_dict = {p.get("id"): p.get("value") for p in d.get("params", []) if isinstance(p, dict)}
                desc = d.get("body", "") or ""
                return params_dict, desc
            elif r.status_code in (403, 429):
                time.sleep(backoff * 4)
            else:
                time.sleep(backoff)
        except Exception:
            time.sleep(backoff)
        backoff *= 1.8
    return {}, ""

# ------------------ CÀO DỮ LIỆU ------------------
all_data = []
offset = 0
limit = PARAMS_BASE["limit"]
page = 1

while True:
    params = {**PARAMS_BASE, "o": offset}
    try:
        r = requests.get(LISTING_URL, params=params, headers=HEADERS, timeout=15)
        if r.status_code != 200:
            print(f"⚠️ Lỗi {r.status_code} tại offset={offset}")
            break
        data = r.json()
        ads = data.get("ads", [])
        total_ads = data.get("total_ads")
        print(f"📄 Trang {page} | Offset {offset} → {len(ads)} tin (Total_ads={total_ads})")
    except Exception as e:
        print(f"⚠️ Lỗi kết nối: {e}")
        break

    if not ads:
        print("Không còn dữ liệu → Dừng.")
        break

    for ad in ads:
        ad_id = ad.get("list_id")
        if not ad_id:
            continue

        price_str = ad.get("price_string", "")
        if not price_str:
            continue

        # --- Lấy chi tiết ---
        params_dict, desc = safe_get_detail(ad_id, HEADERS)
        if not desc:
            desc = ad.get("body", "") or ""

        features = detect_features(desc)

        all_data.append({
            "Tiêu đề": ad.get("subject"),
            "Giá": price_str,
            "Diện tích": f"{ad.get('size')} m²" if ad.get("size") else None,
            "Địa chỉ": f"{ad.get('area_name', '')}, {ad.get('region_name', '')}".strip(", "),
            "Nội thất": params_dict.get("furnishing_rent", params_dict.get("furnishing", "0")),
            **features,
            "Link": f"https://www.nhatot.com/cho-thue-phong-tro/{ad_id}.htm"
        })

        time.sleep(0.3)

    offset += limit
    page += 1

    if total_ads and offset >= total_ads:
        print("🎯 Đã lấy đủ tổng số tin API báo → Dừng.")
        break

    time.sleep(0.8)

# ------------------ LƯU FILE ------------------
df = pd.DataFrame(all_data)
out_file = "nhatot_phongtro_danang_all.xlsx"
df.to_excel(out_file, index=False, engine="openpyxl")

print(f"\n✅ Hoàn tất! Đã lưu {len(df)} tin vào {out_file}")
